In [6]:
import pandas as pd
from pandasql import sqldf

In [7]:
# Downloaded relevant portion for reproducing the queries, the original table consists of 22,756,333 Rows * 712 columns, file size 11.71 GB logical bytes

# SELECT date, country_code, country_name, subregion1_name, cumulative_confirmed, cumulative_deceased, cumulative_recovered, aggregation_level, new_confirmed, new_deceased
# FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
# WHERE country_name="India" OR country_name="United States of America" OR country_name="Italy" OR country_name="France"

# SELECT count(distinct column_name) FROM `bigquery-public-data.covid19_open_data.INFORMATION_SCHEMA.COLUMNS`

df = pd.read_csv("bq-results-20241028-073212-1730100755249.csv")

In [10]:
df.sample(2)

,date,country_code,country_name,subregion1_name,cumulative_confirmed,cumulative_deceased,cumulative_recovered,aggregation_level,new_confirmed,new_deceased
2126958,2020-10-11,US,United States of America,Georgia,1182.0,24.0,NaN,2,1.0,0.0
1910750,2022-06-11,US,United States of America,Indiana,11030.0,NaN,NaN,2,2.0,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4243462 entries, 0 to 4243461
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   date                  object 
 1   country_code          object 
 2   country_name          object 
 3   subregion1_name       object 
 4   cumulative_confirmed  float64
 5   cumulative_deceased   float64
 6   cumulative_recovered  float64
 7   aggregation_level     int64  
 8   new_confirmed         float64
 9   new_deceased          float64
dtypes: float64(5), int64(1), object(4)
memory usage: 323.8+ MB


# Task 1. Total confirmed cases
Build a query that will answer "What was the total count of confirmed cases on Date?" The query needs to return a single row containing the sum of confirmed cases across all countries. The name of the column should be total_cases_worldwide.    
     
Columns to reference:          
cumulative_confirmed    
date   

In [12]:
q = """
SELECT SUM(cumulative_confirmed) AS total_cases_worldwide
FROM df
WHERE date = "2020-04-25"
GROUP BY date
"""
sqldf(q, globals())

,total_cases_worldwide
0,3677991.0


# Task 2. Worst affected areas
Build a query for answering "How many states in the US had more than Death Count deaths on Date?" The query needs to list the output in the field count_of_states.      
Note: Don't include NULL values.      
     
Columns to reference:     
country_name     
subregion1_name (for state information)     
cumulative_deceased    jective.

In [13]:
q = """
SELECT COUNT(DISTINCT t.subregion1_name) AS count_of_states
FROM (
  SELECT subregion1_name
  FROM df
  WHERE date = "2020-04-25"
  AND country_name = "United States of America"
  AND subregion1_name IS NOT NULL
  GROUP BY date, subregion1_name
  HAVING SUM(cumulative_deceased) > 300
) t
"""
sqldf(q, globals())

,count_of_states
0,32


# Task 3. Identify hotspots   
Build a query that will answer "List all the states in the United States of America that had more than Confirmed Cases confirmed cases on Date?" The query needs to return the State Name and the corresponding confirmed cases arranged in descending order. Name of the fields to return state and total_confirmed_cases.     
     
Columns to reference:     
country_code    
subregion1_name (for state information)    
cumulative_confirmed    ective.

In [14]:
q = """
SELECT subregion1_name AS state, SUM(cumulative_confirmed) AS total_confirmed_cases
FROM df
WHERE date = "2020-04-25" AND country_code = "US" AND subregion1_name IS NOT NULL
GROUP BY subregion1_name
HAVING total_confirmed_cases > 2000
ORDER BY total_confirmed_cases DESC;
"""
sqldf(q, globals())

,state,total_confirmed_cases
0,New York,737377.0
1,New Jersey,210489.0
2,Massachusetts,105943.0
3,California,85707.0
4,Illinois,83280.0
5,Pennsylvania,81675.0
6,Michigan,80565.0
7,Florida,60440.0
8,Louisiana,52961.0
9,Georgia,51498.0


# Task 4. Fatality ratio     
Build a query that will answer "What was the case-fatality ratio in Italy for the month of Month 2020?" Case-fatality ratio here is defined as (total deaths / total confirmed cases) * 100.
Write a query to return the ratio for the month of Month 2020 and contain the following fields in the output: total_confirmed_cases, total_deaths, case_fatality_ratio.
     
Columns to reference:      
country_name     
cumulative_confirmed     
cumulative_deceased

In [15]:
q = """
SELECT sum(cumulative_confirmed) as total_confirmed_cases, sum(cumulative_deceased) as total_deaths, (sum(cumulative_deceased)/sum(cumulative_confirmed))*100 as case_fatality_ratio
FROM df
where country_name="Italy" AND date BETWEEN '2020-05-01'and '2020-05-31'
"""
sqldf(q, globals())

,total_confirmed_cases,total_deaths,case_fatality_ratio
0,20315012.0,1947170.0,9.584882


# Task 5. Identifying specific day     
Build a query that will answer: "On what day did the total number of deaths cross Death count in Italy in Italy?" The query should return the date in the format yyyy-mm-dd.
      
Columns to reference:      
country_name      
cumulative_deceased      

In [16]:
q = """
SELECT date
FROM df
WHERE country_name = 'Italy' AND cumulative_deceased > 10000
ORDER BY date LIMIT 1;
"""
sqldf(q, globals())

,date
0,2020-03-28


# Task 6. Finding days with zero net new cases
The following query is written to identify the number of days in India between Start date in India and Close date in India when there were zero increases in the number of confirmed cases. However it is not executing properly.      
You need to update the query to complete it and obtain the result:     

> Not all DBMS support [subqueries using WITH clause](https://www.geeksforgeeks.org/sql-with-clause/); BigQuery does.     
> https://cloud.google.com/bigquery/docs/reference/standard-sql/subqueries

# Task 7. Doubling rate       
Using the previous query as a template, write a query to find out the dates on which the confirmed cases increased by more than 20% compared to the previous day (indicating doubling rate of ~ 7 days) in the US between the dates March 22, 2020 and April 20, 2020. The query needs to return the list of dates, the confirmed cases on that day, the confirmed cases the previous day, and the percentage increase in cases between the days       .       

Use the following names for the returned fields: Date, Confirmed_Cases_On_Day, Confirmed_Cases_Previous_Day and Percentage_Increase_In_Cas       es.

# Task 8. Recovery rate  
Build a query to list the recovery rates of countries arranged in descending order (limit to Limit Value) upto the date May 10, 2020.      
Restrict the query to only those countries having more than 50K confirmed cases.      
The query needs to return the following fields: country, recovered_cases, confirmed_cases, recovery_rate.      
      
Columns to reference:     
* country_name    
* cumulative_confirmed    
* cumulative_recovered    

# Task 9. CDGR - Cumulative daily growth rate
The following query is trying to calculate the CDGR on Date(Cumulative Daily Growth Rate) for France since the day the first case was reported.The first case was reported on Jan 24, 2020.     
     
The CDGR is calculated as:       
((last_day_cases/first_day_cases)^1/days_diff)-1)      
     
Where :     
last_day_cases is the number of confirmed cases on May 10, 2020        
first_day_cases is the number of confirmed cases on Jan 24, 2020       
days_diff is the number of days between Jan 24 - May 10, 2020      
     
The query isn’t executing properly. Can you fix the error to make the query execute successfully?    

> The CDGR is calculated as: ((last_day_cases/first_day_cases)^1/days_diff)-1)      
> https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions

In [21]:
# from math import sqrt
# sqrt(10), pow(10, -1)
# (POWER((last_day_cases/first_day_cases), 1)/days_diff)-1 AS cdgr

# Task 10. Create a Looker Studio report      
Create a Looker Studio report that plots the following for the United States:      
     
Number of Confirmed Cases     
Number of Deaths    
Date range : Date Range     

In [19]:
q = """
SELECT date, SUM(cumulative_confirmed) AS country_cases, SUM(cumulative_deceased) AS country_deaths 
FROM df
WHERE date BETWEEN '2020-03-20' AND '2020-04-23' AND country_name ="United States of America" 
GROUP BY date
"""
sqldf(q, globals())

,date,country_cases,country_deaths
0,2020-03-20,95567.0,1048.0
1,2020-03-21,117806.0,1351.0
2,2020-03-22,144693.0,1717.0
3,2020-03-23,178113.0,2176.0
4,2020-03-24,214851.0,2879.0
5,2020-03-25,257259.0,3734.0
6,2020-03-26,312220.0,4836.0
7,2020-03-27,371694.0,6338.0
8,2020-03-28,429116.0,8033.0
9,2020-03-29,491825.0,9683.0
